---

### **Import the SDK via NUGET**

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview

---
### **Load configuration from secrets.json**

In [3]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeploymentName = apiConfig.deployDavinci; // davinci
//string mydeploymentName = apiConfig.deployTurbo; // turbo
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeploymentName + " & " + myendpoint + " & " + myAPIKey);


---
### **Build the Semantic Kernel client**

In [4]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 


kernel.Config.AddAzureOpenAITextCompletion("davinci", mydeploymentName, myendpoint, myAPIKey);
//kernel.Config.AddAzureOpenAITextCompletion("turbo", mydeploymentName, myendpoint, myAPIKey);
//kernel.Config.AddChatCompletion()


---
### **Simple chat loop**

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SKSkills");

// Load the Chat function from the FunSkill
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");

var myContext = new ContextVariables(); 
var botPrompt = "AI: Hello. What's your name?";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable        
        var input = await InteractiveKernel.GetInputAsync($"{botPrompt} ({(i+1)} of {numberOfRounds})");
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["Chat"]);

        // tack onto the history what's come back from the model
        /********************************************************/
        var theNewChatExchange = $"Me: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 
        /********************************************************/
        // this way the new chat exchange gets passed into the next round

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"AI: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("AI: Thanks for the wonderful chat!");
        break;
    }
}

---
### **Batman Sample Chat**

In [6]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SKSkills");

// Load the Chat function from the 
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkills");

var myContext = new ContextVariables(); 
var botPrompt = "Batman: Hello Batman Fan.  Do you have a question for me?";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable
        Console.WriteLine("👆 Enter text in the input cell above to chat with the bot. 👆\n");
        var input = await InteractiveKernel.GetInputAsync($"{botPrompt} ({(i+1)} of {numberOfRounds})");
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["BatmanChat"]);

        // tack onto the history 👇 what's come back from the model
        /********************************************************/
        var theNewChatExchange = $"Fan: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 
        /********************************************************/
        // this way the new chat exchange gets passed into the next round

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"Batman: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("Batman: Thanks for the wonderful chat!");
        break;
    }
}

👆 Enter text in the input cell above to chat with the bot. 👆

Chat for 1 of 4 rounds with AI:
Batman: Hello Batman Fan.  Do you have a question for me?
Fan: where do you work
AI: I work in Gotham City, protecting its citizens from crime and injustice.

👆 Enter text in the input cell above to chat with the bot. 👆



Error: Input request cancelled

Error: Input request cancelled

Batman: Thanks for the wonderful chat!
